In [6]:
my_models = []

import torchvision.models as models
my_models.append(['resnet50',models.resnet18(pretrained=True)])
# my_models.append(['resnet18',models.resnet18(pretrained=True)])
# # my_models.append(['alexnet', models.alexnet(pretrained=True)])
# my_models.append(['squeezenet', models.squeezenet1_0(pretrained=True)])
# my_models.append(['vgg16', models.vgg16(pretrained=True)])
# my_models.append(['densenet', models.densenet161(pretrained=True)])
# my_models.append(['inception', models.inception_v3(pretrained=True)])
# my_models.append(['googlenet', models.googlenet(pretrained=True)])
# my_models.append(['shufflenet', models.shufflenet_v2_x1_0(pretrained=True)])
# my_models.append(['mobilenet', models.mobilenet_v2(pretrained=True)])
# my_models.append(['resnext50_32x4d', models.resnext50_32x4d(pretrained=True)])
# my_models.append(['wide_resnet50_2', models.wide_resnet50_2(pretrained=True)])
# my_models.append(['mnasnet', models.mnasnet1_0(pretrained=True)])

In [7]:
# CUDA support 
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


In [10]:
from torchsummary  import summary
for i in my_models:
    print('-'*10)
    print(i[0])
    summary(i[1].cuda(), (3, 224, 224))

----------
resnet50
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-1

In [11]:
for i in my_models:
    print('-'*10)
    print(i[0])
    print(i[1])

----------
resnet50
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): R

We will use ResNet50 because according to this article, it is the best:
https://www.learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/
https://arxiv.org/pdf/1512.03385.pdf

Try using it

In [ ]:
import dataLoader3
experimentName = "test_pretrained_model"

# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

from configParser import ConfigParser, getModelName
experiment_params = ConfigParser(experimentName).read()

data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

datasetManager = dataLoader3.datasetManager()
datasetManager.updateParams(experiment_params)
train_loader, validation_loader, test_loader = datasetManager.getLoaders()
dataloaders = {
    'train': train_loader
    'val': test_loader
}

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        N_test = 0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                N_test = N_test + len(inputs)
                inputs = inputs
                labels = labels

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / N_test
            epoch_acc = running_corrects.double() / N_test

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model